<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
%mkdir spark

/


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-21 14:34:09--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.236.52, 140.211.166.134, 64.50.233.100, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.236.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  7.00MB/s    in 2m 26s  

2021-07-21 14:36:36 (1.46 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/




In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark/spark-3.0.3-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
%cd /content/
%mkdir /chapterData 
%cd /chapterData
%mkdir retail-data
%cd retail-data/
%mkdir by-day
%cd by-day

/content
/chapterData
/chapterData/retail-data
/chapterData/retail-data/by-day


In [9]:
%pwd


'/chapterData/retail-data/by-day'

In [12]:

!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-03.csv'

--2021-07-21 14:37:29--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275001 (269K) [text/plain]
Saving to: ‘2010-12-01.csv’

010-12-01.csv       100%[===================>] 268.56K  --.-KB/s    in 0.04s   

2021-07-21 14:37:30 (6.57 MB/s) - ‘2010-12-01.csv’ saved [275001/275001]

--2021-07-21 14:37:30--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected

In [14]:
%mv /chapterData /content/

# New Section

In [11]:
staticDF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("/content/chapterData/retail-data/by-day/*.csv")

AnalysisException: ignored

In [ ]:
staticDF.show(10)

In [ ]:
dayOfLargePurchase = staticDF.groupby("InvoiceDate").sum("UnitPrice").withColumnRenamed("sum(UnitPrice)", "SumOFUnitPrice")
dayOfLargePurchase.sort(dayOfLargePurchase.InvoiceDate.desc()).take(10)

In [ ]:
from pyspark.sql.functions import window , column , desc , col 
staticDataFrame = staticDF.selectExpr("CustomerID", "(UnitPrice*Quantity) as total_purchase_value", "InvoiceDate").groupBy(\
    col("CustomerId"),\
    window(col("InvoiceDate"),"1 day"))\
.sum("total_purchase_value")\
.withColumnRenamed("sum(total_purchase_value)", "total_purchase_value")\
.show(10)

